### Bias classifier 
* Pre-trained models
* bert-base-uncased,facebook/roberta-hate-speech-dynabench-r4-target,T5

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
from transformers import default_data_collator
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset
import pandas as pd
from sklearn.utils import resample
from functools import partial


2025-04-24 10:04:43.937145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745489084.112784      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745489084.162116      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))


CUDA available: True
Device count: 1
Current device: 0
Device name: Tesla P100-PCIE-16GB


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:

pip install --upgrade transformers torch datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 91.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 11.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 6.7 MB/s eta 0:00:000:00:010

In [16]:
import os
import sys
import torch
import numpy as np
import pandas as pd
import cupy as cp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator
)
from functools import partial
import logging

# Set up logging to debug recursion
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Increase recursion limit cautiously
sys.setrecursionlimit(1500)

# Disable W&B
os.environ["WANDB_MODE"] = "disabled"

# Constants
MODEL_NAME = "bert-base-uncased"
LABEL_MAP = {"left": 0, "center": 1, "right": 2}
REVERSE_LABEL_MAP = {v: k for k, v in LABEL_MAP.items()}
OUTPUT_DIR = "/kaggle/working/results"
LOGGING_DIR = "/kaggle/working/logs"
RESULTS_PATH = "/kaggle/working/predictions.csv"
SAVE_PATH = OUTPUT_DIR

# Ensure directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOGGING_DIR, exist_ok=True)
os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# ---------------- Text Processing Functions ----------------
def preprocess_text(text):
    return ' '.join(str(t).lower() for t in text if isinstance(t, str)) if isinstance(text, list) else str(text).lower()

def combine_text(df, text_cols):
    for col in text_cols:
        df[col] = df[col].apply(preprocess_text) if col in df else ""
    df["combined_input"] = df[text_cols].fillna("").agg(" ".join, axis=1)
    return df

def tokenize(batch, tokenizer):
    return tokenizer(batch["combined_input"], padding="max_length", truncation=True, max_length=512)

# ---------------- Rebalancing Function ----------------
def rebalance_three_categories_fixed_length(df, bias_factor=0.35, target_type='center', final_length=None):
    if final_length is None:
        final_length = len(df)

    df_target = df[df['type'] == target_type]
    df_other = df[df['type'] != target_type]

    n_target = int(final_length * bias_factor)
    n_other_total = final_length - n_target
    n_per_other = n_other_total // (df['type'].nunique() - 1)

    if len(df_target) >= n_target:
        df_target_balanced = resample(df_target, n_samples=n_target, replace=False, random_state=42)
    else:
        df_target_balanced = resample(df_target, n_samples=n_target, replace=True, random_state=42)

    other_types = [t for t in df['type'].unique() if t != target_type]
    df_others_balanced = []
    
    for t in other_types:
        df_t = df[df['type'] == t]
        if len(df_t) >= n_per_other:
            df_resampled = resample(df_t, n_samples=n_per_other, replace=False, random_state=42)
        else:
            df_resampled = resample(df_t, n_samples=n_per_other, replace=True, random_state=42)
        df_others_balanced.append(df_resampled)

    df_final = pd.concat([df_target_balanced] + df_others_balanced)
    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
    return df_final

# ---------------- Load & Rebalance ----------------
def load_and_rebalance(file_path):
    logger.info("Loading and rebalancing data")
    df = pd.read_excel(file_path, engine='openpyxl')
    df = df[df["type"].isin(LABEL_MAP.keys())]
    
    logger.info("Original distribution:")
    logger.info(df["type"].value_counts())
    
    df_balanced = rebalance_three_categories_fixed_length(
        df,
        bias_factor=0.35,
        target_type='center',
        final_length=len(df['type'])
    )
    
    logger.info("\nBalanced distribution:")
    logger.info(df_balanced["type"].value_counts())
    
    text_cols = ["text", "topic", "article", "biased_words"]
    df_processed = combine_text(df_balanced, text_cols)
    df_processed["label"] = df_processed["type"].map(LABEL_MAP)
    
    return df_processed

# ---------------- Metrics ----------------
def compute_metrics(eval_pred):
    logger.info("Computing metrics")
    logits, labels = eval_pred
    logits = cp.asarray(logits)
    preds = cp.argmax(logits, axis=1).get()
    labels = labels
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# ---------------- Main Pipeline ----------------
def main(file_path):
    logger.info("Starting main pipeline")
    # Load and prepare data
    df = load_and_rebalance(file_path)
    
    # Train-Test Split
    logger.info("Splitting data")
    train_df, eval_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
    
    # Convert to Dataset objects
    train_ds = Dataset.from_pandas(train_df[["combined_input", "label"]].reset_index(drop=True))
    eval_ds = Dataset.from_pandas(eval_df[["combined_input", "label"]].reset_index(drop=True))

    # Load Tokenizer and Model
    logger.info(f"Loading tokenizer and model: {MODEL_NAME}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=3,
            ignore_mismatched_sizes=True
        )
    except Exception as e:
        logger.error(f"Model {MODEL_NAME} not found, using fallback model. Error: {e}")
        fallback = "facebook/roberta-hate-speech-dynabench-r4-target"
        tokenizer = AutoTokenizer.from_pretrained(fallback)
        model = AutoModelForSequenceClassification.from_pretrained(
            fallback,
            num_labels=3,
            ignore_mismatched_sizes=True
        )

    # Move model to device
    model.to(device)

    # Tokenization
    logger.info("Tokenizing datasets")
    tokenize_fn = partial(tokenize, tokenizer=tokenizer)
    train_ds = train_ds.map(tokenize_fn, batched=True, batch_size=1000)
    eval_ds = eval_ds.map(tokenize_fn, batched=True, batch_size=1000)
    
    train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    eval_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    # Training Arguments
    logger.info("Setting up training arguments")
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        logging_dir=LOGGING_DIR,
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        learning_rate=2e-5,
        weight_decay=0.01,
        remove_unused_columns=False,
        logging_steps=50,
        fp16=torch.cuda.is_available(),
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        report_to="none",
        no_cuda=not torch.cuda.is_available()
    )

    # Initialize Trainer
    logger.info("Initializing Trainer")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
        data_collator=default_data_collator
    )

    # Train
    logger.info("Starting training")
    trainer.train()

    # Save model and tokenizer
    logger.info("Saving model and tokenizer")
    model.save_pretrained(SAVE_PATH)
    tokenizer.save_pretrained(SAVE_PATH)
    logger.info(f"Model and tokenizer saved to {SAVE_PATH}")

    # Prediction
    logger.info("Making predictions")
    full_ds = Dataset.from_pandas(df[["combined_input"]].reset_index(drop=True))
    full_ds = full_ds.map(tokenize_fn, batched=True, batch_size=1000)
    full_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])
    
    preds = trainer.predict(full_ds)
    pred_logits = cp.asarray(preds.predictions)
    pred_labels = cp.argmax(pred_logits, axis=1).get()
    df["predicted_bias_category"] = [REVERSE_LABEL_MAP[i] for i in pred_labels]
    df.to_csv(RESULTS_PATH, index=False)
    logger.info(f"Predictions saved to {RESULTS_PATH}")

if __name__ == "__main__":
    input_file_path = "/kaggle/input/combined-data/combined_data.xlsx"
    main(input_file_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/52884 [00:00<?, ? examples/s]

Map:   0%|          | 0/13221 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.016400,0.010606,0.998563,0.998563,0.998566,0.998563
2,0.005900,0.002697,0.999471,0.999471,0.999471,0.999471
3,0.001000,0.001106,0.999924,0.999924,0.999924,0.999924


Map:   0%|          | 0/66105 [00:00<?, ? examples/s]

### New Sampling techniques

In [2]:
import os
import sys
import torch
import numpy as np
import pandas as pd
import cupy as cp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator
)
from functools import partial
import logging

# Set up logging to debug recursion
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Increase recursion limit cautiously
sys.setrecursionlimit(1500)

# Disable W&B
os.environ["WANDB_MODE"] = "disabled"

# Constants
MODEL_NAME = "bert-base-uncased"
LABEL_MAP = {"left": 0, "center": 1, "right": 2}
REVERSE_LABEL_MAP = {v: k for k, v in LABEL_MAP.items()}
OUTPUT_DIR = "/kaggle/working/results"
LOGGING_DIR = "/kaggle/working/logs"
RESULTS_PATH = "/kaggle/working/predictions.csv"
SAVE_PATH = OUTPUT_DIR

# Ensure directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOGGING_DIR, exist_ok=True)
os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# ---------------- Text Processing Functions ----------------
def preprocess_text(text):
    return ' '.join(str(t).lower() for t in text if isinstance(t, str)) if isinstance(text, list) else str(text).lower()

def combine_text(df, text_cols):
    for col in text_cols:
        df[col] = df[col].apply(preprocess_text) if col in df else ""
    df["combined_input"] = df[text_cols].fillna("").agg(" ".join, axis=1)
    return df

def tokenize(batch, tokenizer):
    return tokenizer(batch["combined_input"], padding="max_length", truncation=True, max_length=512)

# ---------------- Rebalancing Function ----------------
def rebalance_three_categories_fixed_length(df, final_length=None):
    if final_length is None:
        final_length = len(df)

    # Calculate equal number of samples per class
    unique_classes = df['type'].nunique()
    n_per_class = final_length // unique_classes

    # Adjust final_length to ensure it's divisible by number of classes
    final_length = n_per_class * unique_classes

    # Resample each class to have exactly n_per_class samples
    balanced_dfs = []
    for class_type in df['type'].unique():
        df_class = df[df['type'] == class_type]
        if len(df_class) >= n_per_class:
            df_resampled = resample(df_class, n_samples=n_per_class, replace=False, random_state=42)
        else:
            df_resampled = resample(df_class, n_samples=n_per_class, replace=True, random_state=42)
        balanced_dfs.append(df_resampled)

    # Combine and shuffle the balanced dataset
    df_final = pd.concat(balanced_dfs)
    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
    return df_final

# ---------------- Load & Rebalance ----------------
def load_and_rebalance(file_path):
    logger.info("Loading and rebalancing data")
    df = pd.read_excel(file_path, engine='openpyxl')
    df = df[df["type"].isin(LABEL_MAP.keys())]
    
    logger.info("Original distribution:")
    logger.info(df["type"].value_counts())
    
    df_balanced = rebalance_three_categories_fixed_length(
        df,
        final_length=len(df)
    )
    
    logger.info("\nBalanced distribution:")
    logger.info(df_balanced["type"].value_counts())
    
    text_cols = ["text", "topic", "article", "biased_words"]
    df_processed = combine_text(df_balanced, text_cols)
    df_processed["label"] = df_processed["type"].map(LABEL_MAP)
    
    return df_processed

# ---------------- Metrics ----------------
def compute_metrics(eval_pred):
    logger.info("Computing metrics")
    logits, labels = eval_pred
    logits = cp.asarray(logits)
    preds = cp.argmax(logits, axis=1).get()
    labels = labels
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# ---------------- Main Pipeline ----------------
def main(file_path):
    logger.info("Starting main pipeline")
    # Load and prepare data
    df = load_and_rebalance(file_path)
    
    # Train-Test Split
    logger.info("Splitting data")
    train_df, eval_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
    
    # Convert to Dataset objects
    train_ds = Dataset.from_pandas(train_df[["combined_input", "label"]].reset_index(drop=True))
    eval_ds = Dataset.from_pandas(eval_df[["combined_input", "label"]].reset_index(drop=True))

    # Load Tokenizer and Model
    logger.info(f"Loading tokenizer and model: {MODEL_NAME}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=3,
            ignore_mismatched_sizes=True
        )
    except Exception as e:
        logger.error(f"Model {MODEL_NAME} not found, using fallback model. Error: {e}")
        fallback = "facebook/roberta-hate-speech-dynabench-r4-target"
        tokenizer = AutoTokenizer.from_pretrained(fallback)
        model = AutoModelForSequenceClassification.from_pretrained(
            fallback,
            num_labels=3,
            ignore_mismatched_sizes=True
        )

    # Move model to device
    model.to(device)

    # Tokenization
    logger.info("Tokenizing datasets")
    tokenize_fn = partial(tokenize, tokenizer=tokenizer)
    train_ds = train_ds.map(tokenize_fn, batched=True, batch_size=1000)
    eval_ds = eval_ds.map(tokenize_fn, batched=True, batch_size=1000)
    
    train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    eval_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    # Training Arguments
    logger.info("Setting up training arguments")
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        logging_dir=LOGGING_DIR,
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        learning_rate=2e-5,
        weight_decay=0.01,
        remove_unused_columns=False,
        logging_steps=50,
        fp16=torch.cuda.is_available(),
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        report_to="none",
        no_cuda=not torch.cuda.is_available()
    )

    # Initialize Trainer
    logger.info("Initializing Trainer")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
        data_collator=default_data_collator
    )

    # Train
    logger.info("Starting training")
    trainer.train()

    # Save model and tokenizer
    logger.info("Saving model and tokenizer")
    model.save_pretrained(SAVE_PATH)
    tokenizer.save_pretrained(SAVE_PATH)
    logger.info(f"Model and tokenizer saved to {SAVE_PATH}")

    # Prediction
    logger.info("Making predictions")
    full_ds = Dataset.from_pandas(df[["combined_input"]].reset_index(drop=True))
    full_ds = full_ds.map(tokenize_fn, batched=True, batch_size=1000)
    full_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])
    
    preds = trainer.predict(full_ds)
    pred_logits = cp.asarray(preds.predictions)
    pred_labels = cp.argmax(pred_logits, axis=1).get()
    df["predicted_bias_category"] = [REVERSE_LABEL_MAP[i] for i in pred_labels]
    df.to_csv(RESULTS_PATH, index=False)
    logger.info(f"Predictions saved to {RESULTS_PATH}")

if __name__ == "__main__":
    input_file_path = "/kaggle/input/combined-data/combined_data.xlsx"
    main(input_file_path)

2025-04-24 05:38:18.667569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745473098.842827      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745473098.897872      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/52884 [00:00<?, ? examples/s]

Map:   0%|          | 0/13221 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.001600,0.008459,0.998033,0.998033,0.998039,0.998033
2,0.000000,0.001799,0.999622,0.999622,0.999622,0.999622
3,0.000000,0.000734,0.999924,0.999924,0.999924,0.999924


Map:   0%|          | 0/66105 [00:00<?, ? examples/s]

### T5


In [3]:
import os
import sys
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from datasets import Dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator
)
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Constants
MODEL_NAME = "google-t5/t5-base"  # Use "Salesforce/codet5-base" for code-related tasks
LABELS = ["left", "center", "right"]
LABEL_MAP = {"left": 0, "center": 1, "right": 2}
REVERSE_LABEL_MAP = {v: k for k, v in LABEL_MAP.items()}
OUTPUT_DIR = "/kaggle/working/results"
LOGGING_DIR = "/kaggle/working/logs"
RESULTS_PATH = "/kaggle/working/predictions.csv"
SAVE_PATH = OUTPUT_DIR

# Ensure directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOGGING_DIR, exist_ok=True)
os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# ================== TEXT PROCESSING ==================
def preprocess_text(text):
    if isinstance(text, list):
        return ' '.join(str(t).lower() for t in text if isinstance(t, str))
    return str(text).lower()

def combine_text(df, text_cols):
    df[text_cols] = df[text_cols].fillna("").astype(str)
    for col in text_cols:
        df[col] = df[col].apply(preprocess_text)
    df["combined_input"] = df[text_cols].agg(" ".join, axis=1)
    return df

# ================== DATA REBALANCING ==================
def rebalance_three_categories_fixed_length(df, bias_factor=0.35, target_type='center', final_length=None):
    if final_length is None:
        final_length = len(df)

    df_target = df[df['type'] == target_type]
    df_other = df[df['type'] != target_type]

    n_target = int(final_length * bias_factor)
    n_other_total = final_length - n_target
    n_per_other = n_other_total // (df['type'].nunique() - 1)

    df_target_balanced = resample(
        df_target,
        n_samples=n_target,
        replace=len(df_target) < n_target,
        random_state=42
    )

    other_types = [t for t in df['type'].unique() if t != target_type]
    df_others_balanced = []
    
    for t in other_types:
        df_t = df[df['type'] == t]
        df_resampled = resample(
            df_t,
            n_samples=n_per_other,
            replace=len(df_t) < n_per_other,
            random_state=42
        )
        df_others_balanced.append(df_resampled)

    df_final = pd.concat([df_target_balanced] + df_others_balanced)
    return df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# ================== TOKENIZATION ==================
def tokenize(batch, tokenizer):
    inputs = tokenizer(
        batch["combined_input"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    if "label" in batch:
        labels = tokenizer(
            batch["label"],
            padding="max_length",
            truncation=True,
            max_length=10,
            return_tensors="pt"
        )
        inputs["labels"] = labels["input_ids"]
    return inputs

# ================== METRICS ==================
def compute_metrics(eval_pred):
    predictions, label_ids = eval_pred
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    preds = [LABEL_MAP.get(p, -1) for p in decoded_preds if p in LABEL_MAP]
    true_labels = [LABEL_MAP.get(l, -1) for l in decoded_labels if l in LABEL_MAP]
    valid_pairs = [(p, t) for p, t in zip(preds, true_labels) if p != -1 and t != -1]
    if not valid_pairs:
        return {"accuracy": 0.0, "f1": 0.0, "precision": 0.0, "recall": 0.0}
    preds, true_labels = zip(*valid_pairs)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average='weighted')
    acc = accuracy_score(true_labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# ================== MAIN PIPELINE ==================
def main(file_path):
    logger.info("Starting pipeline")
    
    # Load and preprocess data
    df = pd.read_excel(file_path, engine='openpyxl')
    df = df[df["type"].isin(LABELS)]
    df_balanced = rebalance_three_categories_fixed_length(df)
    text_cols = ["text", "topic", "article", "biased_words"]
    df_processed = combine_text(df_balanced, text_cols)
    df_processed["label"] = df_processed["type"]
    
    # Train-test split
    train_df, eval_df = train_test_split(df_processed, test_size=0.2, stratify=df_processed["label"], random_state=42)
    train_ds = Dataset.from_pandas(train_df[["combined_input", "label"]])
    eval_ds = Dataset.from_pandas(eval_df[["combined_input", "label"]])
    
    # Initialize tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    
    # Tokenize datasets
    tokenize_fn = lambda batch: tokenize(batch, tokenizer)
    train_ds = train_ds.map(tokenize_fn, batched=True, batch_size=1000, num_proc=4)
    eval_ds = eval_ds.map(tokenize_fn, batched=True, batch_size=1000, num_proc=4)
    train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    eval_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    
    # Training configuration
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        logging_dir=LOGGING_DIR,
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        learning_rate=2e-5,
        weight_decay=0.01,
        remove_unused_columns=False,
        logging_steps=50,
        fp16=torch.cuda.is_available(),
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        report_to="none",
        no_cuda=not torch.cuda.is_available(),
        gradient_accumulation_steps=4
      )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        compute_metrics=compute_metrics,
        data_collator=default_data_collator
    )
    
    # Train and save
    trainer.train()
    model.save_pretrained(SAVE_PATH)
    tokenizer.save_pretrained(SAVE_PATH)
    
    # Generate predictions with constrained decoding
    full_ds = Dataset.from_pandas(df_processed[["combined_input"]])
    full_ds = full_ds.map(tokenize_fn, batched=True, batch_size=1000, num_proc=4)
    full_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])
    
    # Constrained decoding to ensure predictions are in LABELS
    force_words_ids = tokenizer(LABELS, add_special_tokens=False).input_ids
    with torch.no_grad():
        preds = model.generate(
            input_ids=full_ds["input_ids"].to(device),
            attention_mask=full_ds["attention_mask"].to(device),
            max_length=10,
            force_words_ids=force_words_ids
        )
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    df_processed["predicted_type"] = decoded_preds
    df_processed["predicted_label"] = df_processed["predicted_type"].map(LABEL_MAP)
    df_processed[["combined_input", "type", "predicted_type", "predicted_label"]].to_csv(RESULTS_PATH, index=False)
    logger.info(f"Results saved to {RESULTS_PATH}")

if __name__ == "__main__":
    main("/kaggle/input/combined-data/combined_data.xlsx")

2025-04-25 05:16:52.294907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745558212.482353      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745558212.534018      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Map (num_proc=4):   0%|          | 0/52884 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/13221 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.15 GiB. GPU 0 has a total capacity of 15.89 GiB of which 1.17 GiB is free. Process 2694 has 14.72 GiB memory in use. Of the allocated memory 10.85 GiB is allocated by PyTorch, and 3.58 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)